In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve,f1_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
from tensorflow import keras
from tensorflow.keras import layers
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import pickle
from imblearn.over_sampling import SMOTE 

In [2]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [3]:
# dataset with FRAX CRFs and GRS
with open('ready_whi_sp23', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X1, Y1 = data.get('X', []).values, data.get('Y', []).values
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s1, y_train_s1 = sm.fit_resample(x_train1, y_train1)

In [4]:
# dataset with FRAX CRFs (no grs)
with open('ready_whi_sp23', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X2, Y2 = data.get('X_nogrs', []).values, data.get('Y', []).values
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s2, y_train_s2 = sm.fit_resample(x_train2, y_train2)

In [5]:
# Bayesian optimization for Model 4 (FRAX CRFs + GRS)
def nn_cl_bo(neurons,learning_rate, dropout,
             batch_size, epochs):
    optimizerD= {'Adam':Adam(lr=learning_rate)}
    neurons = round(neurons)
    batch_size = round(batch_size)
    epochs = round(epochs)
    dropout = dropout
    def nn_cl_fun():
        opt = Adam(lr = learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=31, activation='relu'))
        nn.add(Dropout(dropout))
        nn.add(Dense(neurons, activation='relu'))
        nn.add(Dropout(dropout))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer='adam', 
                   metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
    score = cross_val_score(nn, x_train_s1, y_train_s1, 
                            scoring=score_acc, cv=10, 
                            fit_params={'callbacks':[es]}).mean()
    return score

In [6]:
# Set paramaters
params_nn ={
    'neurons': (10, 10000),
    'learning_rate':(0.01, 1),
    'dropout':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100)
}

nn_bo = BayesianOptimization(nn_cl_bo, params_nn, verbose=0,random_state=123)
nn_bo.maximize(init_points=2, n_iter=5)
print("Final result for ANN (GRS) with Bayesian optimization:", nn_bo.max)

120/120 [==============================] - 1s 5ms/step
Final result for ANN (GRS) with Bayesian optimization: {'target': 0.8497106003708893, 'params': {'batch_size': 614.3488210891596, 'dropout': 0.08405376347880598, 'epochs': 42.554580478819304, 'learning_rate': 0.6341016716921827, 'neurons': 2316.678707100004}}


In [9]:
# Bayesian optimization for Model 2 (FRAX CRFs + GRS)
def nn_cl_bo(neurons,learning_rate, dropout,
             batch_size, epochs):
    optimizerD= {'Adam':Adam(lr=learning_rate)}
    neurons = round(neurons)
    batch_size = round(batch_size)
    epochs = round(epochs)
    dropout = dropout
    def nn_cl_fun():
        opt = Adam(lr = learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=30, activation='relu'))
        nn.add(Dropout(dropout))
        nn.add(Dense(neurons, activation='relu'))
        nn.add(Dropout(dropout))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer='adam', 
                   metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
    score = cross_val_score(nn, x_train_s2, y_train_s2, 
                            scoring=score_acc, cv=10, 
                            fit_params={'callbacks':[es]}).mean() 
    return score

In [10]:
# Set paramaters
params_nn ={
    'neurons': (10, 10000),
    'learning_rate':(0.01, 1),
    'dropout':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100)
}
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, verbose=0, random_state=123)
nn_bo.maximize(init_points=2, n_iter=5)
print("Final result for ANN (no GRS) with Bayesian optimization:", nn_bo.max)

400/400 [==============================] - 16s 39ms/step
Final result for ANN (no GRS) with Bayesian optimization: {'target': 0.7016578484997078, 'params': {'batch_size': 726.5190148665778, 'dropout': 0.010074801781291724, 'epochs': 60.29878750573636, 'learning_rate': 0.010630405728982002, 'neurons': 7277.230898601044}}


In [11]:
# grid search for Model 3 (FRAX CRFs + GRS)
params_nn ={
    'neurons': [10,1000,10000],
    'learning_rate':[0.01,0.1,1],
    'dropout':[0.01,0.05,0.1],
    'batch_size':[200,600,1000],
    'epochs':[20,60,100]
}

from sklearn.model_selection import GridSearchCV
def create_model(learning_rate=0.01,dropout=0.0,neurons=1): 
    opt = Adam(lr = learning_rate)
    model = Sequential()
    model.add(Dense(neurons, input_dim=31, kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, 
                        epochs=100, 
                        batch_size=10, verbose=0)

grid=GridSearchCV(estimator=model, param_grid=params_nn,verbose=0,n_jobs=-1,
                  cv=10).fit(x_train_s1, y_train_s1) 
print(grid.best_params_)

{'batch_size': 600, 'dropout': 0.1, 'epochs': 60, 'learning_rate': 1, 'neurons': 10000}


In [12]:
# grid search for Model 1 (FRAX CRFs)
params_nn ={
    'neurons': [10,1000,10000],
    'learning_rate':[0.01,0.1,1],
    'dropout':[0.01,0.05,0.1],
    'batch_size':[200,600,1000],
    'epochs':[20,60,100]
}

from sklearn.model_selection import GridSearchCV
def create_model(learning_rate=0.01,dropout=0.0,neurons=1): 
    opt = Adam(lr = learning_rate)
    model = Sequential()
    model.add(Dense(neurons, input_dim=30, kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, 
                        epochs=100, 
                        batch_size=10, verbose=0)

grid=GridSearchCV(estimator=model, param_grid=params_nn, verbose=0,n_jobs=-1,
                  cv=10).fit(x_train_s2, y_train_s2) 
print(grid.best_params_)

{'batch_size': 600, 'dropout': 0.01, 'epochs': 60, 'learning_rate': 0.1, 'neurons': 10000}
